In [2]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import collections
import os 
from __future__ import print_function
from collections import Counter 

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, core
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
''' ['./Cymbals' './Kicks' './Snares & Claps' './Snares & Claps/Claps'
 './Subbass' './Vocals & Chants']
 '''
dictionary = {}
dictionary["./Kicks"] = 0
dictionary["./Subbass"] = 1
dictionary["./Snares & Claps"] = 2
dictionary["./Snares & Claps/Claps"] = 2
dictionary["./Cymbals"] = 3
print(dictionary)

{'./Snares & Claps': 2, './Kicks': 0, './Cymbals': 3, './Subbass': 1, './Snares & Claps/Claps': 2}


In [4]:
def minMaxNorm(data):
    MAX = np.amax(data)
    MIN = np.amin(data)
    return (data-MIN)/(MAX-MIN)

In [5]:
def listErrors(features, originalIndicies, dic):
    # given array of features, original indicies of errors, label dictionary, return out names of errors and size of labels
    error_lens = []
    len_features = len(features)
    error_labels = features[:, 1].reshape((len_features,1))[originalIndicies]
    
    for key, value in dic.iteritems():
        error_lens.append( (key, len(np.where(error_labels[:] == key)[0])) )
    
    return (features[:, 0].reshape((len_features,1))[originalIndicies], error_lens)

In [6]:
dataDir = 'data'
print(os.listdir(dataDir))
dataSource = dataDir + '/time_series_normalized.npy.npz'
#dataSource = 'data/fft_time_series.npy'

f = np.load(dataSource)
ind = f.files

['.DS_Store', '.floyddata', 'time_series_normalized.npy.npz']


In [ ]:
features = f[ind[0]]

In [ ]:
len_features = len(features)
test_percent = 0.6
testSize = int(len_features*test_percent)
trainSize = len_features - testSize

Y = features[0,1]
X = features[0,2]

print(Y.shape)

# change target from strings -> numbers via dictionary
all_Y = np.zeros((Y.shape[0],), dtype=int)
for i in range(len(Y)):
    all_Y[i] = int (dictionary[Y[i]])  

# Convert into one-hot vectors
num_labels = len(np.unique(all_Y))
print("Number of Labels:")
print(num_labels)
all_Y = np.eye(num_labels)[all_Y]  # One liner trick!
indicies = np.arange(len(all_Y))

train_X, test_X, train_Y, test_Y, indices_train, indices_test = train_test_split(X, all_Y, indicies, test_size=test_percent)

print("Train Set Shape:")
print(train_X.shape)

print("Test Set Shape:")
print(test_X.shape)

print("Train Categories")
c1 = Counter([i for i in Y[indices_train]])
print (c1.items()) 
print("Test Categories")
c1 = Counter([i for i in Y[indices_test]])
print (c1.items()) 
print("Total Categories")
c1 = Counter([i for i in Y[:]])
print (c1.items()) 

In [1]:
batch_size = 32*2
epochs = 2

lstm_nodes = train_X.shape[2]
layer1_nodes = lstm_nodes/2

print('Build model...')
model = Sequential()
model.add(core.Masking(mask_value=0., input_shape=(train_X.shape[1], train_X.shape[2])))
L = LSTM(lstm_nodes, return_sequences = False, dropout=0.2, recurrent_dropout=0.2)
model.add( L )
model.add( core.Dropout(.2))
model.add( Dense(layer1_nodes, activation='sigmoid') )
model.add( Dense(num_labels, activation='softmax') )
#model.add( Dense(num_labels, activation='sigmoid') )

loss_metric = 'categorical_crossentropy'
optimizer_func = 'adam'

# try using different optimizers and different optimizer configs
model.compile(loss=loss_metric, optimizer=optimizer_func, metrics=['accuracy', 'categorical_accuracy'])


print('Train...')
model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs, validation_split=.2, validation_data=(test_X, test_Y))
"""
score, acc1, acc2 = model.evaluate(test_X, test_Y, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy 1:', acc1)
print('Test accuracy 2:', acc2)
"""

NameError: name 'train_X' is not defined

In [ ]:
testPredict = np.argmax(model.predict(test_X), axis = 1)

testGround = np.argmax(test_Y, axis = 1)
errors = np.equal(testPredict, testGround)
errorIndicies = np.where(errors == False)

c1 = Counter([i for i in Y[errorIndicies]])
print (c1.items()) 
c1 = Counter([i for i in Y[indices_test]])
print (c1.items()) 

In [ ]:
def saveModel(weights, init, title):
    # save network 
    for i in range(len(weights)):
        tf.add_to_collection('vars', weights[i])
    saver = tf.train.Saver()
    sess = tf.Session()
    sess.run(init)
    saver.save(sess, title)